In [ ]:
!git clone https://github.com/arunsammit/MPNN-Ptr mpnn_ptr
%cd mpnn_ptr

In [ ]:
!pip install torch-scatter torch-sparse torch-cluster torch-spline-conv torch-geometric -f https://data.pyg.org/whl/torch-1.10.0+cu111.html

In [18]:
from models.mpnn_ptr import *
from models.mpnn import *
from models.seqToseq import *
from torch import nn
import torch
from utils.utils import communication_cost, calculate_baseline

In [9]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [14]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.uniform_(param.data, -0.08, 0.08)

In [5]:
!ls

sample_data


In [10]:
from google.colab import drive
drive.mount('./drive')

Mounted at ./drive


In [11]:
dataloader, distance_matrices = torch.load('drive/MyDrive/data_MTP/data_64.pt')

In [ ]:
dataloader.batch_size

128

In [15]:
max_graph_size = 64
mpnn_ptr = MpnnPtr(input_dim=max_graph_size, embedding_dim=75, hidden_dim=81, K=3, n_layers=4,p_dropout=0, logit_clipping=True, device=device)
mpnn_ptr.to(device)
mpnn_ptr.apply(init_weights)
optim = torch.optim.Adam(mpnn_ptr.parameters(), lr=0.0001)
num_epochs = 3500
epoch_penalty = torch.zeros(len(dataloader))
loss_list_pre = []

In [13]:
mpnn_ptr.train()

MpnnPtr(
  (mpnn): Mpnn(
    (node_init_embedding_layer): Sequential(
      (0): Linear(in_features=64, out_features=75, bias=False)
      (1): ReLU()
    )
    (theta4layers): ModuleList(
      (0): Sequential(
        (0): Linear(in_features=150, out_features=75, bias=False)
        (1): ReLU()
      )
      (1): Sequential(
        (0): Linear(in_features=150, out_features=75, bias=False)
        (1): ReLU()
      )
      (2): Sequential(
        (0): Linear(in_features=150, out_features=75, bias=False)
        (1): ReLU()
      )
    )
    (theta5layers): ModuleList(
      (0): Sequential(
        (0): Linear(in_features=150, out_features=75, bias=False)
        (1): ReLU()
      )
      (1): Sequential(
        (0): Linear(in_features=150, out_features=75, bias=False)
        (1): ReLU()
      )
      (2): Sequential(
        (0): Linear(in_features=150, out_features=75, bias=False)
        (1): ReLU()
      )
    )
    (theta6): Sequential(
      (0): Linear(in_features=75, out_f

In [19]:
mpnn_ptr = torch.load('./drive/MyDrive/data_MTP/model_64.pt')

In [20]:
for epoch in range(num_epochs):
    epoch_penalty[:] = 0
    for i, (data, distance_matrix) in enumerate(zip(dataloader, distance_matrices)):
        num_samples = 16
        
        samples, predicted_mappings, log_likelihoods_sum = mpnn_ptr(data,num_samples)
        # samples shape: (batch_size, num_samples, max_graph_size_in_batch)
        # predicted_mappings shape: (batch_size, max_graph_size_in_batch)
        # log_likelihoods_sum shape: (batch_size,)
        penalty = communication_cost(data.edge_index, data.edge_attr, data.batch, data.num_graphs, distance_matrix, predicted_mappings)
        epoch_penalty[i] = penalty.sum()
        penalty_baseline = calculate_baseline(data.edge_index, data.edge_attr, data.batch, data.num_graphs, distance_matrix, samples)
        loss = torch.mean((penalty.detach() - penalty_baseline.detach()) * log_likelihoods_sum)
        optim.zero_grad()
        loss.backward()
        nn.utils.clip_grad_norm_(mpnn_ptr.parameters(), max_norm=1, norm_type=2)
        optim.step()
    batch_loss = epoch_penalty.sum().item()
    loss_list_pre.append(batch_loss)
    print('Epoch: {}/{}, Loss: {}'.format(epoch + 1, num_epochs, batch_loss))

Epoch: 1455/3500, Loss: 597615040.0
Epoch: 1456/3500, Loss: 602834368.0
Epoch: 1457/3500, Loss: 597316736.0
Epoch: 1458/3500, Loss: 602626496.0
Epoch: 1459/3500, Loss: 633705536.0
Epoch: 1460/3500, Loss: 600915008.0
Epoch: 1461/3500, Loss: 596480640.0
Epoch: 1462/3500, Loss: 598476736.0
Epoch: 1463/3500, Loss: 581659200.0
Epoch: 1464/3500, Loss: 603349376.0
Epoch: 1465/3500, Loss: 602249216.0
Epoch: 1466/3500, Loss: 569025728.0
Epoch: 1467/3500, Loss: 577299584.0
Epoch: 1468/3500, Loss: 600713984.0
Epoch: 1469/3500, Loss: 605289472.0
Epoch: 1470/3500, Loss: 573447808.0
Epoch: 1471/3500, Loss: 582188608.0
Epoch: 1472/3500, Loss: 596497408.0
Epoch: 1473/3500, Loss: 576987712.0
Epoch: 1474/3500, Loss: 562592512.0
Epoch: 1475/3500, Loss: 625937280.0
Epoch: 1476/3500, Loss: 609946048.0
Epoch: 1477/3500, Loss: 590579328.0
Epoch: 1478/3500, Loss: 576181888.0
Epoch: 1479/3500, Loss: 593129472.0
Epoch: 1480/3500, Loss: 608574336.0
Epoch: 1481/3500, Loss: 572056640.0
Epoch: 1482/3500, Loss: 6223

KeyboardInterrupt: ignored

In [ ]:
import matplotlib.pyplot as plt

In [21]:
torch.save(mpnn_ptr,'./drive/MyDrive/data_MTP/model_64.pt')

In [ ]:
fig_pre, ax_pre = plt.subplots()  
ax_pre.plot(loss_list_pre)  
ax_pre.set_xlabel('number of epochs') 
ax_pre.set_ylabel('communication cost')  
ax_pre.set_title("communication cost v/s number of epochs")  